# Bonus 03: Extract a regional mesh-cube with Iris

While GeoVista provides the efficient tools for mesh region extraction, it and Iris know nothing about one another. So, to calculate a regionally-extracted _Iris cube_, GeoVista can do the hard work of determining the subset of cells required, but you must then "reconstruct" an Iris cube from that information. For now, at least, there are no ready-made tools for this (either in Iris or GeoVista).  

The process requires a few steps, which we can summarise as :
  1. Record, on the original global PolyData, the original face indices of each of the cells
  1. Perform extraction (by BBox or otherwise) to get a regional PolyData
  1. Get the face-indices of the selected cells from the regional PolyData  
  1. Index the Iris cube with the selected indices, on the mesh dimension, to extract the regional parts
  1. Construct and attach a suitable Iris mesh to represent the extracted region

(Note: the last step itself is not strictly necessary. It may be sufficent to have a regional data cube with a notional "mesh dimension", but which does not possess an actual Iris mesh.)

---
**First, we just repeat some of the imports and code from 'Sec_05_Region_Extraction'**

to get a global mesh-cube and a PolyData derived from it.

In [ ]:
# Fetch a single cube of test data
from testdata_fetching import lfric_rh_singletime_2d
lfric_rh = lfric_rh_singletime_2d()
#lfric_rh

In [ ]:
# Convert to a PyVista.PolyData
from pv_conversions import pv_from_lfric_cube
pv_global_rh = pv_from_lfric_cube(lfric_rh)
#pv_global_rh

In [ ]:
# Define a lat-lon region to extract
from geovista.geodesic import BBox

lon_min = -15
lon_max = 55
lat_min = -5
lat_max = 35

bbox = BBox(lons=[lon_min, lon_max, lon_max, lon_min], lats=[lat_min, lat_min, lat_max, lat_max])

In [ ]:
# 'Apply' the regiona to the PolyData object
pv_regional_rh = bbox.enclosed(pv_global_rh)
pv_regional_rh

---
## Now the meshcube extraction, as a sequence of steps ...


### Step 1: Add an auxiliary array to our _global_ PolyData
This is to record, for each PolyData cell, the original (face) index which that cell has.

Note: We use numpy.arange() to construct a counting sequence, and make this a new array on the PolyData object, by assigning to a index-name.

In [ ]:
import numpy as np
face_inds = np.arange(pv_global_rh.n_cells)
# Assign PolyData[<a-string>] to create a new attached array.
pv_global_rh.cell_data['original_face_indices'] = face_inds

---

### Step 2: Extract with your Bbox to get a regional PolyData, and show the result.
"Enclose" your global data with the Bounding Box as done in  [Section 05](./Sec_05_RegionExtraction.ipynb) and for demonstration above.

In [ ]:
# 'Apply' the region to the PolyData object.
pv_regional_rh = bbox.enclosed(pv_global_rh)
pv_regional_rh

You can see that the new version of the extracted (regional) data now has an ***extra*** attached data array, "`original_face_indices`", which is derived from the one we added to the global data, and which holds the face indices of the _selected_ cells.

---

### Step 3: Fetch the indices array from the regional PolyData, by indexing with the array name.
and show the result.

In [ ]:
# Get the remaining face indices, to use for indexing the Cube.
region_indices = pv_regional_rh["original_face_indices"]
region_indices

This contains the original face-indices of all the cells which fall within the region, _i.e. which faces those were in the global mesh_.

We can now apply these via indexing, to select only those cells *from the Iris cube*.

### Step 4: Apply these cells as an index to the 'mesh dimension' of the original Iris lfric-rh cube
.. and print that out.

In [ ]:
lfric_rh_region = lfric_rh[..., region_indices]
lfric_rh_region

This new cube contains the mesh data within our selected region.

However, there is a catch here: Once indexed, our cube ***no longer has a mesh***. You can see this in the printout, which lists "Auxiliary coordinates" but no "Mesh coordinates". This problem will probably be fixed in future.  See [here in the Iris docs](https://scitools-iris.readthedocs.io/en/latest/further_topics/ugrid/operations.html#region-extraction) for a discussion. For now, what we need to do is to re-create a mesh for the regional cube.
We do that in a few further steps ...

---

### Step 5a: Get the X and Y-axis coordinates from the region cube.

In [ ]:
x_coord = lfric_rh_region.coord('longitude')
y_coord = lfric_rh_region.coord('latitude')

### Step 5b: Create a new `iris.experimental.ugrid.Mesh`-class object, passing the X,Y coords as arguments

In [ ]:
from iris.experimental.ugrid.mesh import Mesh
mesh = Mesh.from_coords(x_coord, y_coord)

#print(mesh)

---
### Step 5c:  Call `Mesh.to_MeshCoords` to create a pair of `MeshCoord`s containing this mesh
Note: You must specify the keyword `location="face"`:  This matches the data location of the original data, i.e. the data cells are faces.

In [ ]:
mesh_coords = mesh.to_MeshCoords(location="face")
mesh_coords

---
### Step 5d : Use `Cube.remove_coord` and `Cube.add_aux_coord` to replace each AuxCoord with its corresponding `MeshCoord` from the previous step. 
Note: For 'add_aux_coord', you also need to specify the relevant cube dimension(s). See Iris documentation for [Cube.add_aux_coord](https://scitools-iris.readthedocs.io/en/latest/generated/api/iris/cube.html?highlight=add_aux_coord#iris.cube.Cube.add_aux_coord)  
.. and show the cube ..

In [ ]:
lfric_rh_region.remove_coord('longitude')
lfric_rh_region.remove_coord('latitude')

In [ ]:
xco, yco = mesh_coords

lfric_rh_region.add_aux_coord(xco, 0)
lfric_rh_region.add_aux_coord(yco, 0)

# Result : a regional Mesh-Cube with a subset of the original faces.
lfric_rh_region

---

## Plot this to see what we got.
Use the techniques as above, converting with `pv_from_lfric_cube` and plotting.


In [ ]:
pv = pv_from_lfric_cube(lfric_rh_region)
pv.plot()

----

**Investigation:** It is useful to add some extra background information to make this more visible.

As a minimum you can use `plotter.add_coastlines()`.

Another useful one is `plotter.add_base_layer()`  
**Question :  what does that actually do ?**

In [ ]:
from geovista import GeoPlotter
plotter = GeoPlotter()
plotter.add_mesh(pv)
plotter.add_coastlines()
plotter.add_base_layer()
plotter.show()